UI to work with SBI stuff

In [1]:
import os
import subprocess
import io
from ipykernel.iostream import OutStream
import sys

def source_lmod_script(script_path):
    """
    Source an Lmod/module script and import environment variables into Python safely,
    suppressing terminal warnings.
    """
    # Use a non-interactive login shell (bash -l), redirect errors
    command = f'bash -l -c "source {script_path} >/dev/null 2>&1; printenv -0"'
    
    proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    out, _ = proc.communicate()
    
    # Parse null-separated environment variables
    for env_var in out.split(b'\0'):
        if env_var:
            key, _, value = env_var.partition(b'=')
            os.environ[key.decode()] = value.decode()

# Example usage
M3_BUILD_DIR = "/home/henryi/scratch/venvs/.venv_dune/bin/"
source_lmod_script(f"{M3_BUILD_DIR}/setup.MaCh3.sh")
source_lmod_script(f"{M3_BUILD_DIR}/setup.MaCh3DUNE.sh")
os.environ["OMP_NUM_THREADS"] = "8"


In [2]:
from mach3sbitools.ui.sbi_ui import MaCh3SbiUI
from mach3sbitools.utils.device_handler import TorchDeviceHander

from pathlib import Path

from sbi.utils import BoxUniform

/scratch/henryi/venvs/.venv_dune/lib/python3.11/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


Found MaCh3 DUNE instance, using that!
Couldn't find Tutorial instance
FOUND


In [ ]:
input_file = Path("/home/henryi/dunesft/MaCh3_DUNE/Configs/EventRates_Beam.yaml")
ui = MaCh3SbiUI(str(input_file), 'DUNE')
device_handler = TorchDeviceHander()

In [ ]:
# Fit settings!
FIT_TYPE='FastEpsFree'
N_ROUNDS = 20
SAMPLES_PER_ROUND= 10

fitter_settings = {}

training_settings = {}



In [ ]:
# Now we can run the fit!
ui.run_fit(FIT_TYPE, N_ROUNDS, SAMPLES_PER_ROUND, fitter_settings, training_settings)

In [ ]:
s = ui.fitter.posterior.sample((100000,), device_handler.to_tensor(ui.fitter.x0))
from matplotlib import pyplot as plt
ui.fitter.save(Path('my_fit.pkl'))

In [ ]:
import uproot as ur
import pickle
from matplotlib import pyplot as plt

tree = ur.open("/home/henryi/sft/MaCh3Tutorial/Test.root:posteriors")

with open('/home/henryi/sft/MaCh3SbiTools/fitter.pkl', 'rb') as handle:
    saved_posterior = pickle.load(handle)

s = saved_posterior.sample((100000,), device_handler.to_tensor(ui.fitter.x0))

arrs = tree.arrays(library="np")


In [ ]:

for i in range(len(s[0])):    
    plt.hist(s[10000:,i].cpu(), alpha=0.3, label="ML", bins=100, density=True)

    key_name = list(arrs.keys())[i]
    plt.hist(arrs[key_name][100000:], alpha=0.3, label="MCMC", bins=100, density=True)
    plt.legend()
    plt.xlabel(key_name)
    plt.show()

In [ ]:
print(s)